In [6]:
# Importer les bibliothèques nécessaires
import pandas as pd
pd.set_option('display.max_columns', 375)
import numpy as np
from category_encoders import CountEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [3]:
# Charger les données
print("Chargement des données...")
X_train = pd.read_csv('~/data/train_input_Z61KlZo.csv') 
y_train = pd.read_csv('~/data/train_output_DzPxaPY.csv')
# X_test = pd.read_csv("test_input.csv")
print("Données chargées avec succès.")

Chargement des données...


/tmp/ipykernel_17790/2952621689.py:3: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv('~/data/train_input_Z61KlZo.csv')


Données chargées avec succès.


In [4]:
# Traitement des valeurs manquantes dans les colonnes numériques
print("Traitement des valeurs manquantes dans les colonnes numériques...")
numeric_columns = X_train.select_dtypes(include=['number']).columns

# Remplir les NaN avec 0 pour les colonnes numériques
X_train[numeric_columns] = X_train[numeric_columns].fillna(0)

# Identifier les colonnes non numériques
fill_cols = [item for item in X_train.columns if item not in numeric_columns]

# Remplir les NaN des colonnes non numériques avec une valeur par défaut (-999)
X_train[fill_cols] = X_train[fill_cols].fillna(-999)

print("Traitement des valeurs manquantes terminé.")

Traitement des valeurs manquantes dans les colonnes numériques...
Traitement des valeurs manquantes terminé.


In [8]:
# Préparation des données pour l'entraînement
print("Préparation des données pour l'entraînement...")

# Suppression des colonnes inutiles
X_train = X_train.drop(['ID', 'ANNEE_ASSURANCE'], axis=1)

# Encodage des variables catégoriques avec CountEncoder
encoder = CountEncoder(cols=fill_cols)
encoder.fit(X_train)
X_train_enc = encoder.transform(X_train)

print("Préparation terminée.")

Préparation des données pour l'entraînement...
Préparation terminée.


In [ ]:
# Entraîner les modèles
print("Entraînement des modèles")

# Modèle pour prédire 'FREQ'
rf_freq = RandomForestRegressor(random_state=42)
rf_freq.fit(X_train_enc, y_train['FREQ'])
print("Modèle pour 'FREQ' entraîné avec succès.")


Entraînement des modèles


In [7]:
# Prédire 'FREQ' sur l'ensemble d'entraînement
y_train_pred_freq = rf_freq.predict(X_train_enc)

In [8]:
# Modèle pour prédire 'CM'
gb_cm = GradientBoostingRegressor(random_state=42)
gb_cm.fit(X_train_enc, y_train['CM'])
print("Modèle pour 'CM' entraîné avec succès.")

Modèle pour 'CM' entraîné avec succès.


In [9]:
# Prédire 'CM' sur l'ensemble d'entraînement
y_train_pred_cm = gb_cm.predict(X_train_enc)

In [10]:
# Calculer la prédiction combinée pour 'CHARGE'
y_train_pred = y_train_pred_freq * y_train_pred_cm * y_train['ANNEE_ASSURANCE']

In [11]:
# Calculer le RMSE sur l'ensemble d'entraînement
rmse = np.sqrt(mean_squared_error(y_train['CHARGE'], y_train_pred))
print(f"RMSE sur l'ensemble d'entraînement : {rmse:.2f}")

RMSE sur l'ensemble d'entraînement : 6613.76


In [12]:
# Traitement des données de test
print("Traitement des données de test...")

# Remplir les valeurs manquantes
X_test[numeric_columns] = X_test[numeric_columns].fillna(0)
X_test[fill_cols] = X_test[fill_cols].fillna(-999)

# Suppression des colonnes inutiles
X_test_model = X_test.drop(['ID', 'ANNEE_ASSURANCE'], axis=1)

# Transformation des données de test avec le même encoder
X_test_enc = encoder.transform(X_test_model)

print("Traitement des données de test terminé.")

Traitement des données de test...
Traitement des données de test terminé.


In [13]:
# Prédictions sur l'ensemble de test
print("Prédictions sur l'ensemble de test...")

# Prédire 'FREQ' et 'CM' sur les données de test
y_pred_freq = rf_freq.predict(X_test_enc)
y_pred_cm = gb_cm.predict(X_test_enc)

Prédictions sur l'ensemble de test...


In [14]:
# Combiner les prédictions
y_pred = pd.concat([
    X_test[['ID', 'ANNEE_ASSURANCE']].reset_index(drop=True),
    pd.DataFrame(y_pred_freq, columns=['FREQ']),
    pd.DataFrame(y_pred_cm, columns=['CM'])
], axis=1)

In [15]:
# Calculer la colonne 'CHARGE' en multipliant 'FREQ', 'CM' et 'ANNEE_ASSURANCE'
y_pred['CHARGE'] = y_pred['CM'] * y_pred['FREQ'] * y_pred['ANNEE_ASSURANCE']

print("Prédictions terminées.")

Prédictions terminées.


In [16]:
# Exporter les prédictions dans un fichier CSV
print("Exportation des résultats...")
y_pred.to_csv('submission.csv', index=False)
print("Fichier de soumission créé : 'submission.csv'")

Exportation des résultats...
Fichier de soumission créé : 'submission.csv'
